In [121]:
import pandas as pd
import altair as alt

In [122]:
dados_federais_campina = pd.read_csv("data\\dados_federais_obitos_cg.csv", encoding='utf-8', error_bad_lines=False)
dados_painelfederal_campina = pd.read_csv("data\\dados_painel-federal_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Processando os dados sobre Obitos Confirmados por Dia

In [123]:
# Função que cria os dataframes e 'seta' a coluna 'data' como index
def criar_dataframe(dataframe, nome_coluna_data, nome_coluna_obitos):
    df = pd.DataFrame({
        'data': dataframe[nome_coluna_data].tolist(),
        'obitos_confirmados': dataframe[nome_coluna_obitos].tolist()
    })

    # Setando a coluna 'data' como índice do DataFrame 
    df.set_index('data', inplace=True)
    return df

 ## Painel Federal

In [124]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
pf_obitosPorDia = dados_painelfederal_campina[['obitosNovos', 'data']]


In [125]:
# Criando um DataFrame com os dados para o altair
pf_obitosPorDia = criar_dataframe(pf_obitosPorDia, 'data', 'obitosNovos')

# Criando uma nova coluna no DataFrame referente a média movel.
pf_obitosPorDia['media_movel'] = pf_obitosPorDia.obitos_confirmados.rolling(7).mean()

## Dados Federais

In [126]:
# Criando um data com a coluna 'dataNotificacao' e a coluna 'saidaConfirmadaObitos'
df_obitosPorDia = dados_federais_campina[['dataNotificacao', 'saidaConfirmadaObitos']]

#### OBS: Os dados federais estão descritos pela data e cada data diluida pelo estabelecimento de saúde ao qual foi notificado.

In [127]:
# Agrupando as informações de 'saidaConfirmadaObitos' pela data de notificação
df_obitosPorDia = df_obitosPorDia.groupby('dataNotificacao')
df_obitosPorDia = df_obitosPorDia['saidaConfirmadaObitos'].apply(list)
df_obitosPorDia = df_obitosPorDia.reset_index()

# Somando os valores que foram agrupados pela data
for i in df_obitosPorDia.index:
    df_obitosPorDia.loc[df_obitosPorDia.index == i, 'saidaConfirmadaObitos'] = sum(df_obitosPorDia.iloc[i]['saidaConfirmadaObitos'])


In [128]:

# Criando um DataFrame com os dados para o altair
df_obitosPorDia = criar_dataframe(df_obitosPorDia, 'dataNotificacao', 'saidaConfirmadaObitos')

# Criando uma nova coluna no DataFrame referente a média movel.
df_obitosPorDia['media_movel'] = df_obitosPorDia.obitos_confirmados.rolling(7).mean()

## Dados Municipais

In [129]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
# OBS: 'data' já é o index
dm_obitosPorDia = dados_municipais_campina['obitosPorDia']

Como as datas dos dados municipais já estão como index, é preciso criar o DataFrame manualmente

In [130]:
# Criando um DataFrame com os dados para o altair
dm_obitosPorDia = pd.DataFrame({
    'data': dm_obitosPorDia.index.tolist(),
    'obitos_confirmados': dm_obitosPorDia.values.tolist()
})

# Criando uma nova coluna no DataFrame referente a média movel.
dm_obitosPorDia['media_movel'] = dm_obitosPorDia.obitos_confirmados.rolling(7).mean()

# Setando a coluna 'data' como índice do DataFrame 
dm_obitosPorDia.set_index('data', inplace=True)


# Excluindo as datas não contidas nos DataFrames

In [131]:
# Função que exclui os elementos (pelo index) do DataFrame que não estão em outro DataFrame
def exclui_nao_contidos(df_conjunto, df_subconjunto):
    for i in df_conjunto.index:
        if i not in df_subconjunto.index:
            df_conjunto = df_conjunto.drop(i)
    
    return df_conjunto

 ## Painel Federal e Federal

In [132]:
# Excluindo as datas que estão nos dados do painel federal mas não estão nos dados federais
pf_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia, df_obitosPorDia)

# Excluindo as datas que estão nos dados federais mas não estão nos dados do painel federal
df_obitosPorDia = exclui_nao_contidos(df_obitosPorDia, pf_obitosPorDia)

## Painel Federal e Municipal

In [133]:
# Excluindo as datas que estão nos dados federais mas não estão nos dados municipais
pf_obitosPorDia = exclui_nao_contidos(pf_obitosPorDia, dm_obitosPorDia)

# Excluindo as datas que estão nos dados municipais mas não estão nos dados federais
df_obitosPorDia = exclui_nao_contidos(df_obitosPorDia, dm_obitosPorDia)

# Estatística descritiva dos Dados

In [134]:
def estatistica_descritiva(pf, df, dm, col_name):
    print("[PF] -> Painel Federal\n[DF] -> Dados Federais\n[DM] -> Dados Municipais")
    print("Intervalo de 2021-06-01 até 2021-08-31")
    print("     média     mediana  variância   DP")
    print("[PF] %.2f       %.2f      %.2f      %.2f" % (pf[col_name].mean(), 
                                                        pf[col_name].median(), 
                                                        pf[col_name].var(), 
                                                        pf[col_name].std()))

    print("[DF] %.2f       %.2f      %.2f      %.2f" % (df[col_name].mean(), 
                                                        df[col_name].median(), 
                                                        df[col_name].var(), 
                                                        df[col_name].std()))
    
    print("[DM] %.2f       %.2f      %.2f      %.2f" % (dm[col_name].mean(), 
                                                        dm[col_name].median(), 
                                                        dm[col_name].var(), 
                                                        dm[col_name].std()))

In [135]:
estatistica_descritiva(pf_obitosPorDia, df_obitosPorDia, dm_obitosPorDia, 'obitos_confirmados')

[PF] -> Painel Federal
[DF] -> Dados Federais
[DM] -> Dados Municipais
Intervalo de 2021-06-01 até 2021-08-31
     média     mediana  variância   DP
[PF] 2.12       2.00      4.31      2.08
[DF] 2.52       2.00      4.96      2.23
[DM] 2.70       3.00      6.10      2.47


# Transformando os DataFrame do formato wide para o formato long

In [136]:
# Função que transforma os DataFrames do formato wide para o formato long
def to_longFormat(dt_painelfederal, dt_federal, dt_municipal, nome_coluna):
    long_df = pd.DataFrame({
    'data': dt_federal.index.tolist(),
    'Painel Federal': dt_painelfederal[nome_coluna].tolist(),
    'Federal': dt_federal[nome_coluna].tolist(),
    'Municipal': dt_municipal[nome_coluna].tolist()
    }).melt(
        id_vars=['data'], 
        value_vars=['Painel Federal', 'Federal', 'Municipal'],
        var_name='fonte_dados', 
        value_name='obitos_por_dia'
    )
    return long_df

In [137]:
# Transformando os DataFrames sobre obitos confirmados
obitosPorDia = to_longFormat(pf_obitosPorDia, df_obitosPorDia, dm_obitosPorDia, 'obitos_confirmados')

# Transformando os DataFrames sobre a média movel dos obitos confirmados
obitosPorDiaMedMovel = to_longFormat(pf_obitosPorDia, df_obitosPorDia, dm_obitosPorDia, 'media_movel')

# Comparando os dados do Painel Federal, Federais e Municipais

In [138]:
# Função que cria os graficos de linhas (mark_line) no altair
def criar_mark_line(dataframe, titulo, pontos):
    mark_line = alt.Chart(dataframe).mark_line(point=pontos).encode(
                    alt.X('data:T', title='Data'),
                    alt.Y('obitos_por_dia:Q', title='Nº de Obitos'),
                    alt.Color('fonte_dados:N', title='Base dos Dados')
                ).properties(
                    width=1000,
                    title=titulo
                )
    return mark_line

### Obitos Confirmados por Dia

In [139]:
criar_mark_line(obitosPorDia, 'Quantidade de Obitos Confirmados por Dia', False)

alt.Chart(...)

### Média Movel (7 dias) dos Obitos Confirmados

In [140]:
criar_mark_line(obitosPorDiaMedMovel, 'Média Movel de Obitos Confirmados por Dia', True)


alt.Chart(...)

### Total de Obitos Confirmados

In [141]:
pf_obitosTotais = pf_obitosPorDia['obitos_confirmados'].sum()
df_obitosTotais = df_obitosPorDia['obitos_confirmados'].sum()
dm_obitosTotais = dm_obitosPorDia['obitos_confirmados'].sum()

In [142]:
totalObitos = pd.DataFrame({
    'base': ['Painel Federal', 'Federal', 'Municipal'],
    'total': [pf_obitosTotais, df_obitosTotais, dm_obitosTotais]
})

In [143]:
alt.Chart(totalObitos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Obitos Confirmados'),
    alt.Color('base:N', title='Base de Dados')
).properties(
    width=250,
    title='Total de Obitos Confirmados (Comparação)'
)

alt.Chart(...)